In [63]:
import os
def create_directory_recursively(path):
    try:
        # exist_ok=True 옵션은 해당 경로의 폴더가 이미 존재해도 에러를 발생시키지 않음
        os.makedirs(path, exist_ok=True)
        # print(f"'{path}' 디렉토리가 생성되었거나 이미 존재합니다.")
    except PermissionError:
        # 권한 오류 처리
        print(f"권한 오류: '{path}' 디렉토리를 생성할 수 없습니다. 접근 권한을 확인하세요.")
    except Exception as e:
        # 기타 오류 처리
        print(f"오류 발생: {e}")
def find_files(root_directory, partial_name):
    """
    주어진 루트 디렉토리에서 부분 이름을 포함하는 모든 파일의 경로를 찾습니다.

    :param root_directory: 검색을 시작할 루트 디렉토리
    :param partial_name: 찾고자 하는 파일 이름의 일부
    :return: 찾은 파일의 전체 경로 목록
    """
    matching_files = []
    for root, dirs, files in os.walk(root_directory):
        for file in files:
            if partial_name in file:
                matching_files.append(os.path.join(root, file))
    return matching_files
def is_point_inside_bbox(bbox, point):
    """
    주어진 bbox 내에 특정 좌표가 포함되어 있는지 확인합니다.

    :param bbox: Bounding box의 좌표 (x_min, y_min, x_max, y_max)
    :param point: 확인할 좌표 (x, y)
    :return: 좌표가 bbox 내에 있으면 True, 아니면 False
    """
    x_min, y_min, x_max, y_max = bbox
    x, y = point

    return x_min <= x <= x_max and y_min <= y <= y_max
def filter_tracks_by_label(annotations, labels):
    """
    주어진 라벨에 해당하는 트랙들만 필터링합니다.

    Parameters:
    - annotations: 'dict_data['annotations']["track"]' 형태의 리스트
    - labels: 필터링할 라벨들의 리스트

    Returns:
    - filtered_tracks: 주어진 라벨들에 해당하는 트랙들의 리스트
    """
    filtered_tracks = [track for track in annotations if track["@label"] in labels]
    return filtered_tracks

# 필터링할 라벨들
labels_to_filter = [
    "Pelvis",
    "Center head",
    "Left hip",
    "Left knee",
    "Left foot",
    "Right  hip",  # 'Right hip' 오타 수정
    "Right knee",
    "Right foot",
    "Spine naval",
    "Spine chest",
    "Neck base",
    "Right shoulder",
    "Right elbow",
    "Right hand",
    "Left shoulder",
    "Left elbow",
    "Left hand"
]
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def draw_bbox_with_keypoints(keypoints):
    """
    주어진 키포인트들을 포함하는 바운딩 박스를 그립니다.

    Parameters:
    - keypoints: [(x1, y1), (x2, y2), ..., (x17, y17)] 형태의 키포인트 리스트
    """

    # 키포인트들로부터 x와 y 좌표 분리
    x_coords = [kp[0] for kp in keypoints]
    y_coords = [kp[1] for kp in keypoints]
    
    # 바운딩 박스의 최소 x, 최대 x, 최소 y, 최대 y 계산
    min_x = min(x_coords)
    max_x = max(x_coords)
    min_y = min(y_coords)
    max_y = max(y_coords)
    
    # # 바운딩 박스의 폭과 높이 계산
    # width = max_x - min_x
    # height = max_y - min_y

    # # 바운딩 박스 및 키포인트 그리기
    # fig,ax = plt.subplots(1)
    # ax.scatter(x_coords, y_coords)  # 키포인트 그리기
    # bbox = patches.Rectangle((min_x, min_y), width, height, linewidth=1, edgecolor='r', facecolor='none')
    # ax.add_patch(bbox)
    # plt.show()
    return  min_x, min_y, max_x, max_y
def group_points_by_frame(filtered_tracks):
    """
    필터링된 트랙들에서 동일 프레임의 키포인트들을 리스트로 묶습니다.

    Parameters:
    - filtered_tracks: 필터링된 트랙들의 리스트

    Returns:
    - grouped_points: 프레임별 키포인트 리스트의 딕셔너리
    """
    frame_points = {}

    # 모든 트랙과 포인트를 순회하면서 프레임별로 키포인트 그룹화
    for track in filtered_tracks:
            point=    track["points"][0]
            frame = point["@frame"]
            # 각 포인트를 (x, y) 형태로 변환
            point_tuple =  tuple([int(x)  for x in point["@points"].split(",")])
            if frame not in frame_points:
                frame_points[frame] = [point_tuple]
            else:
                frame_points[frame].append(point_tuple)

    # 동일 프레임 내의 키포인트들을 정렬하여 튜플로 변환
    grouped_points = {frame: sorted(points, key=lambda x: x[0]) for frame, points in frame_points.items()}

    return grouped_points

In [51]:
import os
def count_mp4_files(path,end):
  """
  주어진 경로와 하위 폴더들을 재귀적으로 탐색하여 mp4 파일 개수를 센다.

  Args:
    path: 탐색을 시작할 경로

  Returns:
    mp4 파일 개수
  """
  mp4_files = []
  mp4_count = 0
  def _count_mp4_files_in_dir(dir_path):
    nonlocal mp4_count
    nonlocal mp4_files
    for file in os.listdir(dir_path):
      full_path = os.path.join(dir_path, file)
      if os.path.isfile(full_path):
        if file.endswith(f".{end}") and not "매장이동" in full_path:
          mp4_count += 1
          mp4_files.append(full_path)
      elif os.path.isdir(full_path):
        _count_mp4_files_in_dir(full_path)

  _count_mp4_files_in_dir(path)

  return mp4_count, mp4_files

# 예시 코드
path = "/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/01-1.정식개방데이터"
xml_count, xml_files = count_mp4_files(path,"xml")
print(xml_count)

4670


In [3]:
import json
import xmltodict



In [22]:
import pickle
data=[]
with open("output.pkl", 'wb') as file2:
    pickle.dump(data, file2)
with open("output.pkl", 'rb') as file:
    data = pickle.load(file)
print(data)

[]


In [71]:
import copy
import pickle
import numpy as np  
root_directory = "/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/PreProcessToPkl" # 검색을 시작할 루트 디렉토리 경로로 변경하세요.
error_list=[]
action_list= [
    "select_start","select_end",
    "test_start", "test_end",
    "buying_start", "buying_end",
    "return_start", "return_end",
    "compare_start", "compare_end"
    ]
for idx, xml_file in enumerate(xml_files):
    print("-------------------",idx)
    with open(xml_file, 'r', encoding='utf-8') as file:
        xml_string = file.read()
        # XML 문자열을 파이썬 딕셔너리로 변환
    dict_data = xmltodict.parse(xml_string)
    actionbbox_list=[]
    for i in dict_data['annotations']["track"]:
        if i["@label"] in action_list:
            if "start" in i["@label"]:
                if(type(i['box'])==list):
                    bbox = (i["@label"] , i['box'][0]["@frame"] ,(float(i['box'][0]["@xtl"]),float(i['box'][0]["@ytl"]),float(i['box'][0]["@xbr"]),float(i['box'][0]["@ybr"])))
                    actionbbox_list.append(bbox)
                else:
                    bbox = (i["@label"] , i['box']["@frame"] ,(float(i['box']["@xtl"]),float(i['box']["@ytl"]),float(i['box']["@xbr"]),float(i['box']["@ybr"])))
                    actionbbox_list.append(bbox)
    # annotations 데이터 예시 (사용자의 실제 데이터를 여기에 맞게 조정 필요)
    annotations = dict_data['annotations']["track"]

    # 필터링 실행
    filtered_annotations = filter_tracks_by_label(annotations, labels_to_filter)


    # 필터링된 트랙을 사용하여 함수 호출
    grouped_points_by_frame = group_points_by_frame(filtered_annotations)

    # 결과 확인
    # for frame, points in grouped_points_by_frame.items():
    #     print(len(points))
    #     print(f"Frame {frame}: {points}")
    bbox_list=[]
    for frame, points in grouped_points_by_frame.items():
        min_x, min_y, max_x, max_y = draw_bbox_with_keypoints(points)
        bbox_list.append( (min_x, min_y, max_x, max_y))
    pkl_path = copy.deepcopy(xml_file)
    pklsplit=pkl_path.split("/")[-1][:-4]
    found_files = find_files(root_directory, pklsplit)
    if(len(found_files)!=len(actionbbox_list)):
        print(len(found_files))
        print(found_files)
        error_list.append(found_files)
        print(len(actionbbox_list))
        print(actionbbox_list)
        continue
    for idx , found_file in enumerate(found_files):
        with open(found_file, 'rb') as file:
            data = pickle.load(file)
            newfilepath=found_file.replace("PreProcessToPkl","PreProcessToPkl3")
            if(os.path.exists(newfilepath)):
                continue
            if(data["keypoint"].shape[0]==1):
                continue
            data_keypoints = list(data["keypoint"].transpose((1, 0, 2, 3)))
            data_keypoint_scores = list(data["keypoint_score"].transpose((1, 0, 2)))
            new_data_keypoint_list=[]
            new_data_keypoint_score_list=[]
            print(found_files)
            print(data["keypoint"].shape)
            
            for item in zip(data_keypoints,data_keypoint_scores,bbox_list):
                data_keypoint,data_keypoint_score,bbox= item
                points = list(data_keypoint)
                scores = list(data_keypoint_score)
                for idx2, item2 in enumerate(zip(points,scores)):
                    point ,score=item2
                    for m in list(point):
                        if is_point_inside_bbox(bbox, m):
                            point = np.expand_dims(point,axis=0)
                            score = np.expand_dims(score,axis=0)
                            new_data_keypoint_list.append(point)
                            new_data_keypoint_score_list.append(score)
                            
                            break
                    
            new_video_dir = os.path.dirname(newfilepath)
            create_directory_recursively(new_video_dir)
            new_data_keypoint=np.array(new_data_keypoint_list)
            print(new_data_keypoint.shape)
            new_data_keypoint_score=np.array(new_data_keypoint_score_list)
            print(new_data_keypoint_score.shape)
            data["keypoint"] = new_data_keypoint.transpose((1, 0, 2, 3))
            data["keypoint_score"] = new_data_keypoint_score.transpose((1, 0, 2))
            print("바뀐keypoint:",data["keypoint"].shape)
            with open(newfilepath, 'wb') as file2:
                pickle.dump(data, file2)
        break


------------------- 0
------------------- 1
------------------- 2
------------------- 3
------------------- 4
------------------- 5
------------------- 6
------------------- 7
------------------- 8
------------------- 9
------------------- 10
------------------- 11
------------------- 12
------------------- 13
------------------- 14
------------------- 15
------------------- 16
------------------- 17
------------------- 18
------------------- 19
------------------- 20
------------------- 21
------------------- 22
------------------- 23
------------------- 24
------------------- 25
------------------- 26
------------------- 27
------------------- 28
------------------- 29
------------------- 30
------------------- 31
------------------- 32
------------------- 33
------------------- 34
------------------- 35
------------------- 36
------------------- 37
------------------- 38
------------------- 39
------------------- 40
------------------- 41
------------------- 42
------------------- 4

KeyboardInterrupt: 

In [ ]:
with open(found_file, 'rb') as file:
    data = pickle.load(file)

In [13]:
with open('/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/PreProcessToPkl/Validation/01.원천데이터/VS_02.구매행동_06.비교/C_2_6_55_BU_DYB_10-23_15-27-36_CA_RGB_DF1_F3_F3_116_243.pkl', 'rb') as file:
    data = pickle.load(file)
data["keypoint"].shape

(7, 128, 17, 2)

In [76]:
print(data["keypoint"].shape)
print(data["keypoint"][0].shape)



(1, 24, 17, 2)
(24, 17, 2)
[758.3477  411.09702]


In [71]:
tuple(data["keypoint"][0][0][0])
frame,label,bbox =actionbbox_list[0]
bbox

(486.67, 223.33, 975.0, 1031.7)

In [72]:

# 사용 예시
point = tuple(data["keypoint"][0][0][0])  # 확인할 좌표

if is_point_inside_bbox(bbox, point):
    print("주어진 좌표는 bbox 내에 있습니다.")
else:
    print("주어진 좌표는 bbox 밖에 있습니다.")


주어진 좌표는 bbox 내에 있습니다.


In [23]:
erro_xml_list=[]
erro_video_list=[]
with open("output_xml_error.pkl", 'rb') as file2:
    erro_xml_list =pickle.load(file2)
with open("output_video_error.pkl", 'rb') as file2:
    erro_video_list =pickle.load(file2)


In [25]:
len(erro_video_list)

5

In [26]:
len(erro_xml_list)

5

In [27]:
erro_xml_list

['/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/01-1.정식개방데이터/Validation/02.라벨링데이터/VL_02.구매행동_05.반품/C_2_5_113_BU_DYB_10-20_16-00-20_CA_RGB_DF1_M4_M4.xml',
 '/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/01-1.정식개방데이터/Training/02.라벨링데이터/TL_02.구매행동_05.반품/C_2_5_40_BU_DYB_10-10_13-40-04_CB_RGB_DF1_F2_F2.xml',
 '/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/01-1.정식개방데이터/Training/02.라벨링데이터/TL_02.구매행동_05.반품/C_2_5_14_BU_DYB_10-10_11-39-08_CC_RGB_DF1_F1_F1.xml',
 '/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/01-1.정식개방데이터/Training/02.라벨링데이터/TL_02.구매행동_05.반품/C_2_5_1_BU_DYB_10-10_11-25-04_CC_RGB_DF1_M1_M1.xml',
 '/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/01-1.정식개방데이터/Training/02.라벨링데이터/TL_02.구매행동_05.반품/C_2_5_33_BU_DYB_10-10_12-49-14_CA_RGB_DF1_M2_M2.xml']

In [38]:
import copy
import pickle
import numpy as np  
root_directory = "/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/PreProcessToPkl" # 검색을 시작할 루트 디렉토리 경로로 변경하세요.
error_list=[]
action_list= [
    "select_start","select_end",
    "test_start", "test_end",
    "buying_start", "buying_end",
    "return_start", "return_end",
    "compare_start", "compare_end"
    ]
for idx, xml_file in enumerate(xml_files):
    print("-------------------",idx)
    print(xml_file)
    with open(xml_file, 'r', encoding='utf-8') as file:
        xml_string = file.read()
        # XML 문자열을 파이썬 딕셔너리로 변환
    dict_data = xmltodict.parse(xml_string)
    actionbbox_list=[]
    for i in dict_data['annotations']["track"]:
        if i["@label"] in action_list:
            if "start" in i["@label"]:
                if(type(i['box'])==list):
                    bbox = (i["@label"] , i['box'][0]["@frame"] ,(float(i['box'][0]["@xtl"]),float(i['box'][0]["@ytl"]),float(i['box'][0]["@xbr"]),float(i['box'][0]["@ybr"])))
                    actionbbox_list.append(bbox)
                else:
                    bbox = (i["@label"] , i['box']["@frame"] ,(float(i['box']["@xtl"]),float(i['box']["@ytl"]),float(i['box']["@xbr"]),float(i['box']["@ybr"])))
                    actionbbox_list.append(bbox)
    pkl_path = copy.deepcopy(xml_file)
    pklsplit=pkl_path.split("/")[-1][:-4]
    found_files = find_files(root_directory, pklsplit)
    if(len(found_files)!=len(actionbbox_list)):
        print(len(found_files))
        print(found_files)
        error_list.append(found_files)
        print(len(actionbbox_list))
        print(actionbbox_list)
        continue
    for idx , found_file in enumerate(found_files):
        with open(found_file, 'rb') as file:
            data = pickle.load(file)
            newfilepath=found_file.replace("PreProcessToPkl","PreProcessToPkl3")
            if(os.path.exists(newfilepath)):
                continue
            if(data["keypoint"].shape[0]==1):
                continue
            keypoint = np.zeros((30, 1, 17, 2),dtype=np.float16)
            data["keypoint"] = data["keypoint"].transpose((1, 0, 2, 3))
            for idx2,j in enumerate(data["keypoint"]):
                frame,label,bbox =actionbbox_list[idx]
                point = tuple(j[0][0])
                if is_point_inside_bbox(bbox, point):
                    new_video_dir = os.path.dirname(newfilepath)
                    create_directory_recursively(new_video_dir)
                    data["keypoint"] = np.expand_dims(j,axis=0)
                    # with open(newfilepath, 'wb') as file2:
                        # pickle.dump(data, file2)
                        # pass
                else:
                    continue
        break
    break


------------------- 0
/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/01-1.정식개방데이터/Validation/02.라벨링데이터/VL_02.구매행동_06.비교/C_2_6_55_BU_DYB_10-23_15-27-36_CB_RGB_DF1_F3_F3.xml


In [39]:
xml_file="/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/01-1.정식개방데이터/Validation/02.라벨링데이터/VL_02.구매행동_06.비교/C_2_6_55_BU_DYB_10-23_15-27-36_CB_RGB_DF1_F3_F3.xml"
with open(xml_file, 'r', encoding='utf-8') as file:
    xml_string = file.read()
    # XML 문자열을 파이썬 딕셔너리로 변환
dict_data = xmltodict.parse(xml_string)
actionbbox_list=[]
for i in dict_data['annotations']["track"]:
    if i["@label"] in action_list:
        if "start" in i["@label"]:
            if(type(i['box'])==list):
                bbox = (i["@label"] , i['box'][0]["@frame"] ,(float(i['box'][0]["@xtl"]),float(i['box'][0]["@ytl"]),float(i['box'][0]["@xbr"]),float(i['box'][0]["@ybr"])))
                actionbbox_list.append(bbox)
            else:
                bbox = (i["@label"] , i['box']["@frame"] ,(float(i['box']["@xtl"]),float(i['box']["@ytl"]),float(i['box']["@xbr"]),float(i['box']["@ybr"])))
                actionbbox_list.append(bbox)
print(actionbbox_list)


[('compare_start', '116', (893.3, 260.0, 1215.0, 728.3)), ('compare_start', '447', (879.54, 276.41, 1179.31, 733.59))]


In [ ]:
Right foot  ,  Spine chest

In [7]:
import xmltodict
xml_file="/home/bigdeal/mnt2/238-1.실내(편의점,_매장)_사람_구매행동_데이터/01-1.정식개방데이터/Validation/02.라벨링데이터/VL_02.구매행동_06.비교/C_2_6_55_BU_DYB_10-23_15-27-36_CB_RGB_DF1_F3_F3.xml"
with open(xml_file, 'r', encoding='utf-8') as file:
    xml_string = file.read()
    # XML 문자열을 파이썬 딕셔너리로 변환
dict_data = xmltodict.parse(xml_string)
all_list=[]
batch_list = []
print(dict_data['annotations']["track"][0]["@label"])
count=0

for i in dict_data['annotations']["track"]:

    if(i["@label"]=="Right foot"):
        batch_list.clear()
    batch_list.append(i)
    if(i["@label"]=="Spine chest"):
        all_list.append(batch_list)
        break
    
    count+=1
print(all_list)
print(len(all_list))
print(len(batch_list))

object_groceries
[[{'@id': '8', '@label': 'Right foot', '@source': 'manual', 'points': [{'@frame': '116', '@outside': '0', '@occluded': '0', '@keyframe': '1', '@points': '908,728', '@z_order': '0', 'attribute': {'@name': 'ID', '#text': '0'}}, {'@frame': '117', '@outside': '1', '@occluded': '0', '@keyframe': '1', '@points': '908,728', '@z_order': '0', 'attribute': {'@name': 'ID', '#text': '0'}}]}, {'@id': '9', '@label': 'Right knee', '@source': 'manual', 'points': [{'@frame': '116', '@outside': '0', '@occluded': '0', '@keyframe': '1', '@points': '932,649', '@z_order': '0', 'attribute': {'@name': 'ID', '#text': '0'}}, {'@frame': '117', '@outside': '1', '@occluded': '0', '@keyframe': '1', '@points': '932,649', '@z_order': '0', 'attribute': {'@name': 'ID', '#text': '0'}}]}, {'@id': '10', '@label': 'Right  hip', '@source': 'manual', 'points': [{'@frame': '116', '@outside': '0', '@occluded': '0', '@keyframe': '1', '@points': '968,560', '@z_order': '0', 'attribute': {'@name': 'ID', '#text': '

In [29]:
import numpy as np
kepoint_list=[]
for i in batch_list:
    kepoint_list.append([int(x)  for x in i["points"][0]["@points"].split(",")])
min_x, min_y, max_x, max_y = draw_bbox_with_keypoints(kepoint_list)
print( min_x, min_y, max_x, max_y)

[[908, 728], [932, 649], [968, 560], [1048, 570], [1028, 657], [1014, 727], [1010, 562], [1054, 385], [967, 524], [932, 498], [978, 402], [1118, 387], [1136, 478], [1164, 503], [1081, 327], [1018, 522], [1038, 437]]
908 327 1164 728


In [46]:


# annotations 데이터 예시 (사용자의 실제 데이터를 여기에 맞게 조정 필요)
annotations = dict_data['annotations']["track"]

# 필터링 실행
filtered_annotations = filter_tracks_by_label(annotations, labels_to_filter)

# 결과 출력 (필요에 따라)

print(len(filtered_annotations))


4403


In [49]:


# 필터링된 트랙을 사용하여 함수 호출
grouped_points_by_frame = group_points_by_frame(filtered_annotations)

# 결과 확인
# for frame, points in grouped_points_by_frame.items():
#     print(len(points))
#     print(f"Frame {frame}: {points}")
for frame, points in grouped_points_by_frame.items():
    min_x, min_y, max_x, max_y = draw_bbox_with_keypoints(points)
    print( min_x, min_y, max_x, max_y)

17
Frame 116: [(908, 728), (932, 649), (932, 498), (967, 524), (968, 560), (978, 402), (1010, 562), (1014, 727), (1018, 522), (1028, 657), (1038, 437), (1048, 570), (1054, 385), (1081, 327), (1118, 387), (1136, 478), (1164, 503)]
17
Frame 117: [(910, 725), (933, 648), (934, 503), (967, 562), (973, 533), (977, 402), (1008, 562), (1014, 727), (1014, 520), (1027, 657), (1033, 435), (1048, 570), (1048, 385), (1078, 324), (1110, 385), (1123, 475), (1157, 487)]
17
Frame 118: [(909, 731), (932, 649), (937, 510), (966, 562), (973, 401), (981, 536), (1004, 563), (1012, 519), (1013, 726), (1026, 655), (1026, 436), (1041, 383), (1044, 569), (1072, 324), (1102, 383), (1118, 473), (1149, 478)]
17
Frame 119: [(910, 725), (929, 649), (940, 518), (963, 566), (971, 406), (994, 540), (1001, 566), (1009, 522), (1015, 725), (1022, 438), (1025, 654), (1040, 382), (1041, 569), (1071, 322), (1097, 384), (1106, 470), (1138, 470)]
17
Frame 120: [(910, 725), (928, 647), (941, 523), (962, 568), (967, 411), (1000